### Metoda `apply`

Metoda apply jest często używaną metodą, która upraszcza stosowanie dowolnej funkcji na każdym elemencie serii oraz na każdej kolumnie lub wierszu ramki.

Bardzo poglądowo mówiąc, możemy powiedzieć, że "wektoryzuje" wskazaną przez nas funkcję. Poglądowo, ponieważ nie pociąga to za sobą zwiększenia wydajności obliczeniowej, charakterystycznego dla operacji zwektoryzowanych. Ale sam mechanizm działania wygląda tak, jakbyśmy zastosowali wskazaną funkcję na wszystkich elementach kolekcji. 

#### Seria

Stwórzmy przykładową serię zawierającą imiona studentów i ich wzrost w cm.

In [1]:
import pandas as pd
import numpy as np


students = pd.Series(
    data=[180, 175, 168, 190],
    index=['Vik', 'Mehdi', 'Bella', 'Chriss']
)
students

Vik       180
Mehdi     175
Bella     168
Chriss    190
dtype: int64

Teraz napiszmy funkcję, która przelicza centymetry na stopy.

In [2]:
def cm_to_feet(h):
    return round(h / 30.48, 2)

Sprawdźmy jej działanie.

In [3]:
x = 180

cm_to_feet(x)

5.91

Jeżeli spróbujemy teraz zastosować naszą funkcję na serii zadziała, ale tylko dzięki temu, że wszystkie operacje które wykorzystujemy w naszej funkcji ma swoje zwektoryzowane odpowiedniki dla ramki.

In [4]:
cm_to_feet(students)

Vik       5.91
Mehdi     5.74
Bella     5.51
Chriss    6.23
dtype: float64

Wystarczy jednak użyć w naszej funkcji cokolwiek, co nie będzie obsługiwane przez ramkę (a przez float będzie), żeby nasza funkcja przestała działać na ramkach.

In [5]:
import numpy as np

def cm_to_feet(h):
    if h != np.nan:
        return np.round(h / 30.48, 2)

In [6]:
cm_to_feet(x)

5.91

In [7]:
cm_to_feet(students)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

W obecnej postaci nasza funkcja działa na floatach (zgodnie z zamierzeniem), ale nie działa na serii, ponieważ serie nie potrafią porównywać się z `np.nan`.

Gdybyśmy jednak zastosowali naszą funkcję na poszczególnych elementach serii to ten problem by nas nie dotyczył (ponieważ w funkcji będą lądowały poszczególne elementy serii, które są floatami).

Moglibyśmy to zrobić w pętli iterując po wszystkich elementach serii i przykładając naszę funkcję do każdego z elementów. Albo możemy użyć funkcji `apply`, która zrobi to za nas.

In [8]:
students.apply(cm_to_feet)

Vik       5.91
Mehdi     5.74
Bella     5.51
Chriss    6.23
dtype: float64

#### Ramka

A jak używać metody `apply` do przetwarzania kolumn i wierszy w ramce ?

Na potrzeby prezentacji stwórzmy przykładową ramkę zawierającą dane pracowników fikcyjnej firmy.

In [9]:
data = pd.DataFrame({
    'EmployeeName': ['Callen Dunkley', 'Sarah Rayner', 'Jeanette Sloan', 'Kaycee Acosta', 'Henri Conroy', 'Emma Peralta', 'Martin Butt', 'Alex Jensen', 'Kim Howarth', 'Jane Burnett'],
    'Department': ['Accounting', 'Engineering', 'Engineering', 'HR', 'HR', 'HR', 'Data Science', 'Data Science', 'Accounting', 'Data Science'],
    'HireDate': [2010, 2018, 2012, 2014, 2014, 2018, 2020, 2018, 2020, 2012],
    'Sex': ['M', 'F', 'F', 'F', 'M', 'F', 'M', 'M', 'M', 'F'],
    'Birthdate': ['04/09/1982', '14/04/1981', '06/05/1997', '08/01/1986', '10/10/1988', '12/11/1992', '10/04/1991', '16/07/1995', '08/10/1992', '11/10/1979'],
    'Weight': [78, 80, 66, 67, 90, 57, 115, 87, 95, 57],
    'Height': [176, 160, 169, 157, 185, 164, 195, 180, 174, 165],
    'Kids': [2, 1, 0, 1, 1, 0, 2, 0, 3, 1]
})
data

,EmployeeName,Department,HireDate,Sex,Birthdate,Weight,Height,Kids
0,Callen Dunkley,Accounting,2010,M,04/09/1982,78,176,2
1,Sarah Rayner,Engineering,2018,F,14/04/1981,80,160,1
2,Jeanette Sloan,Engineering,2012,F,06/05/1997,66,169,0
3,Kaycee Acosta,HR,2014,F,08/01/1986,67,157,1
4,Henri Conroy,HR,2014,M,10/10/1988,90,185,1
5,Emma Peralta,HR,2018,F,12/11/1992,57,164,0
6,Martin Butt,Data Science,2020,M,10/04/1991,115,195,2
7,Alex Jensen,Data Science,2018,M,16/07/1995,87,180,0
8,Kim Howarth,Accounting,2020,M,08/10/1992,95,174,3
9,Jane Burnett,Data Science,2012,F,11/10/1979,57,165,1


##### Funkcja `apply` w działaniu na kolumnę ramki.

Przejdźmy przez kilka scenariuszy

**Scenariusz I**

Załóżmy, że zespół HR chce wysłać zaproszenie e-mail, które zaczyna się od przyjaznego powitania dla wszystkich pracowników (np. Hej, Sarah!). Poprosili nas o utworzenie dwóch kolumn do przechowywania imion i nazwisk pracowników osobno, co ułatwi odwoływanie się do imion pracowników. W tym celu możemy użyć funkcji lambda, która podzieli ciąg na listę po rozdzieleniu go za pomocą określonego separatora.

In [10]:
# Zwróć uwagę! Metoda `apply` jest wykonywana na serii i to seria jest przekazywana
# do funkcji wewnętrznej
"Ala Kowalska".split()

['Ala', 'Kowalska']

In [12]:
def get_first_name(fullname):
    return fullname.split()[0]

def get_last_name(fullname):
    return fullname.split()[-1]

data['EmployeeName'].apply(get_first_name)

0      Callen
1       Sarah
2    Jeanette
3      Kaycee
4       Henri
5        Emma
6      Martin
7        Alex
8         Kim
9        Jane
Name: EmployeeName, dtype: object

In [13]:
data['FirstName'] = data['EmployeeName'].apply(get_first_name)
data['LastName'] = data['EmployeeName'].apply(get_first_name)
data

,EmployeeName,Department,HireDate,Sex,Birthdate,Weight,Height,Kids,FirstName,LastName
0,Callen Dunkley,Accounting,2010,M,04/09/1982,78,176,2,Callen,Callen
1,Sarah Rayner,Engineering,2018,F,14/04/1981,80,160,1,Sarah,Sarah
2,Jeanette Sloan,Engineering,2012,F,06/05/1997,66,169,0,Jeanette,Jeanette
3,Kaycee Acosta,HR,2014,F,08/01/1986,67,157,1,Kaycee,Kaycee
4,Henri Conroy,HR,2014,M,10/10/1988,90,185,1,Henri,Henri
5,Emma Peralta,HR,2018,F,12/11/1992,57,164,0,Emma,Emma
6,Martin Butt,Data Science,2020,M,10/04/1991,115,195,2,Martin,Martin
7,Alex Jensen,Data Science,2018,M,16/07/1995,87,180,0,Alex,Alex
8,Kim Howarth,Accounting,2020,M,08/10/1992,95,174,3,Kim,Kim
9,Jane Burnett,Data Science,2012,F,11/10/1979,57,165,1,Jane,Jane


In [23]:
# funkcja anonimowa

# def square(x):
#     return x ** 2

square = lambda x: x ** 2
square(3)

9

In [27]:
# rozwiazanie z funkcja anonimowa

data['EmployeeName'].apply(lambda fullname: fullname.split()[0])

0      Callen
1       Sarah
2    Jeanette
3      Kaycee
4       Henri
5        Emma
6      Martin
7        Alex
8         Kim
9        Jane
Name: EmployeeName, dtype: object

In [28]:
data['EmployeeName'].apply(lambda fullname: fullname.split()[-1])

0    Dunkley
1     Rayner
2      Sloan
3     Acosta
4     Conroy
5    Peralta
6       Butt
7     Jensen
8    Howarth
9    Burnett
Name: EmployeeName, dtype: object

**Scenariusz II**

Załóżmy teraz, że zespół HR chce znać wiek każdego pracownika oraz średni wiek pracowników, ponieważ chcą ustalić, czy wiek pracownika wpływa na zadowolenie z pracy i zaangażowanie w pracę.

Aby to zrobić, pierwszym krokiem jest zdefiniowanie funkcji, która pobiera datę urodzenia pracownika i zwraca jego wiek:

In [16]:
from datetime import datetime, date

def calculate_age(birthday):
    birthday_dt = datetime.strptime(birthday, '%d/%m/%Y')
    today = date.today()
    return today.year - birthday_dt.year - (today.month < birthday_dt.month)

In [17]:
calculate_age('04/09/1982')

42

In [18]:
calculate_age('04/12/1982')

41

I teraz możemy zastosować funkcję `calculate_age` na kolumnie `Birthdate` ramki za pomocą metody `apply`.

In [19]:
# Zwróć uwagę! Znów metoda `apply` jest wykonywana na rzecz serii.
data['Birthdate'].apply(calculate_age)

0    42
1    43
2    27
3    38
4    36
5    31
6    33
7    29
8    32
9    45
Name: Birthdate, dtype: int64

Teraz łatwo możemy policzyć średni wiek pracowników.

In [20]:
data['Birthdate'].apply(calculate_age).mean()

35.6

In [21]:
data['Age'] = data['Birthdate'].apply(calculate_age)
data

,EmployeeName,Department,HireDate,Sex,Birthdate,Weight,Height,Kids,FirstName,LastName,Age
0,Callen Dunkley,Accounting,2010,M,04/09/1982,78,176,2,Callen,Callen,42
1,Sarah Rayner,Engineering,2018,F,14/04/1981,80,160,1,Sarah,Sarah,43
2,Jeanette Sloan,Engineering,2012,F,06/05/1997,66,169,0,Jeanette,Jeanette,27
3,Kaycee Acosta,HR,2014,F,08/01/1986,67,157,1,Kaycee,Kaycee,38
4,Henri Conroy,HR,2014,M,10/10/1988,90,185,1,Henri,Henri,36
5,Emma Peralta,HR,2018,F,12/11/1992,57,164,0,Emma,Emma,31
6,Martin Butt,Data Science,2020,M,10/04/1991,115,195,2,Martin,Martin,33
7,Alex Jensen,Data Science,2018,M,16/07/1995,87,180,0,Alex,Alex,29
8,Kim Howarth,Accounting,2020,M,08/10/1992,95,174,3,Kim,Kim,32
9,Jane Burnett,Data Science,2012,F,11/10/1979,57,165,1,Jane,Jane,45


**Scenariusz III**

Kierownik HR w firmie bada opcje ubezpieczenia zdrowotnego dla wszystkich pracowników. Potencjalni dostawcy wymagają informacji o pracownikach. Żałóżmy, że kierownik HR poprosił cię o obliczenie wskaźnika masy ciała (BMI) dla każdego pracownika, aby mogła uzyskać oferty od potencjalnych dostawców usług zdrowotnych.

Aby wykonać to zadanie, najpierw musimy zdefiniować funkcję, która oblicza wskaźnik masy ciała (BMI). Wzór na BMI to waga w kilogramach podzielona przez wzrost w metrach do kwadratu. Ponieważ wzrost pracowników jest mierzony w centymetrach, musimy podzielić wzrost przez 100, aby uzyskać wzrost w metrach.

In [ ]:
...

Zastosujmy naszą funkcję na poszczególnych pracownikach.

In [ ]:
# Zwróć uwagę, tym razem metoda `apply` wykonywana jest na rzecz całej ramki 
# argument axis=1 powoduje, że do funkcji wewnętrznej przekazywane są kolejne
# wiersze ramki, gdyby przekazano axis=0 do funkcji wewnętrznej przekazywane
# byłyby kolejne kolumny ramki.
# i dlatego to cała ramka przekazywana jest do funkcji wewnętrznej.
...

Funkcja lambda pobiera wartości wagi i wzrostu z każdego wiersza, a następnie stosuje funkcję `calc_bmi`, aby obliczyć dla nich wskaźnik masy ciała (BMI). Argument `axis=1` oznacza, że metoda `apply` iteruje po wierszach ramki.

Ostatnim krokiem jest zaklasyfikowanie pracowników według pomiaru BMI. BMI poniżej 18,5 to *Group One*, od 18,5 do 24,9 to *Group Two*, od 25 do 29,9 to *Group Three*, a powyżej 30 to *Group Four*.

In [ ]:
...

In [ ]:
# Zwróć uwagę! Metoda apply jest tutaj stosowana na serii.
...

#### Scenatiusz IV

Załóżmy, że nowy rok zbliża się wielkimi krokami, a zarząd firmy ogłosił, że pracownicy z ponad dziesięcioletnim doświadczeniem otrzymają dodatkowy bonus. Kierownik HR chce wiedzieć, kto kwalifikuje się do otrzymania bonusu.

In [ ]:
...

In [ ]:
...

Ale tutaj uwaga! Chociaż przykład dobrze prezentuje użycie metody `apply`, to w tym przypadku znacznie bardziej wydajnym obliczeniowo i właściwym podejściem jest zastosowanie zwykłego filtrowania. Ze względów wydajnościowych jeżeli można gdzieś zastąpić metodę apply indeksowaniem/filtrowaniem to należy to zrobić. 

In [ ]:
...